# Examples from Chapter 5 — MCP Tools

## Setup Instructions

To ensure you have the required dependencies to run this notebook, you'll need to have our `llm-agents-from-scratch` framework installed on the running Jupyter kernel. To do this, you can launch this notebook with the following command while within the project's root directory:

```sh
uv run --with jupyter jupyter lab
```

Alternatively, if you just want to use the published version of `llm-agents-from-scratch` without local development, you can install it from PyPi by uncommenting the cell below.

In [ ]:
# Uncomment the line below to install `llm-agents-from-scratch` from PyPi
# !pip install llm-agents-from-scratch

## Running an Ollama service

To execute the code provided in this notebook, you’ll need to have Ollama installed on your local machine and have its LLM hosting service running. To download Ollama, follow the instructions found on this page: https://ollama.com/download. After downloading and installing Ollama, you can start a service by opening a terminal and running the command `ollama serve`.

## The Hailstone MCP server
The examples in this notebook demonstrate how to use the framework's MCP integration using a toy MCP server that exposes the familiar Hailstone tool.

**Important:** Run this notebook from within the project's root directory.

The code for the MCP server is located at: https://github.com/nerdai/llm-agents-from-scratch/tree/main/extra/mcp-hailstone

## Examples

### Example 1: Creating an `MCPToolProvider`

In [1]:
from llm_agents_from_scratch.tools.mcp import MCPToolProvider
from mcp import StdioServerParameters
from pathlib import Path

server_path = Path.cwd().parent / "extra/mcp-hailstone"
server_params = StdioServerParameters(
    command="uv",
    args=["run", "--with", "mcp", "mcp", "run", "main.py"],
    cwd=server_path,
)
provider = MCPToolProvider(
    name="hailstone",
    stdio_params=server_params,
)

In [2]:
provider

### Example 2: Establishing a session (connection) to Hailstone MCP server

In [3]:
import time

provider = MCPToolProvider(
    name="hailstone",
    stdio_params=server_params,
)

start = time.perf_counter()
await provider.session()
print(f"First call (establish): {time.perf_counter() - start:.6f}s")

start = time.perf_counter()
await provider.session()
print(f"Second call (cached): {time.perf_counter() - start:.6f}s")

First call (establish): 0.293290s
Second call (cached): 0.000052s


### Example 3: Discovering tools and creating `MCPTool` representations

In [4]:
provider = MCPToolProvider(
    name="hailstone",
    stdio_params=server_params,
)

tools = await provider.get_tools()
print(f"Number of tools discovered: {len(tools)}")

Number of tools discovered: 1


### Example 4: Tearing down the session

In [5]:
provider = MCPToolProvider(
    name="hailstone",
    stdio_params=server_params,
)

await provider.session()
print(f"Session is None: {provider._session is None}")
print(f"Session is ready: {provider._session_ready.is_set()}")

print("-------------")

await provider.close()
print(f"Session is None: {provider._session is None}")
print(f"Session is ready: {provider._session_ready.is_set()}")

Session is None: False
Session is ready: True
-------------
Session is None: True
Session is ready: False
